In [ ]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# def match_histograms(src, ref):
#     src_values, src_lookup, src_counts = np.unique(src.reshape(-1),
#                                                     return_inverse=True,
#                                                     return_counts=True)
#     tmpl_values, tmpl_counts = np.unique(ref.reshape(-1),
#                                             return_counts=True)
#     src_count, src_bins = np.histogram(src.reshape(-1), 255)
#     src_quantiles = np.cumsum(src_counts) / src.size
#     tmpl_quantiles = np.cumsum(tmpl_counts) / ref.size

#     interp_a_values = np.interp(src_quantiles, tmpl_quantiles, tmpl_values)
#     return interp_a_values[src_lookup].reshape(src.shape)

# def match_histograms(src, ref):
#     src_values, src_lookup, src_counts = np.unique(src.reshape(-1),
#                                                     return_inverse=True,
#                                                     return_counts=True)
#     tmpl_values, tmpl_counts = np.unique(ref.reshape(-1),
#                                             return_counts=True)
#     src_quantiles = np.cumsum(src_counts) / src.size
#     src_quantiles = np.insert(src_quantiles, 0, [0]*src_values[0]) 
#     src_quantiles = np.append(src_quantiles, [1]*(255-src_values[-1]))

#     tmpl_quantiles = np.cumsum(tmpl_counts) / ref.size
#     tmpl_quantiles = np.insert(tmpl_quantiles, 0, [0]*tmpl_values[0]) 
#     tmpl_quantiles = np.append(tmpl_quantiles, [1]*(255-tmpl_values[-1])) 

#     interp_a_values = np.interp(src_quantiles, tmpl_quantiles, tmpl_values)
#     return interp_a_values[src.ravel()].reshape(src.shape)

def match_histograms(src, ref):
    src_lookup = src.reshape(-1)
    src_counts = np.bincount(src_lookup)
    tmpl_counts = np.bincount(ref.reshape(-1))

    # omit values where the count was 0
    tmpl_values = np.nonzero(tmpl_counts)[0]
    tmpl_counts = tmpl_counts[tmpl_values]
    
    src_cdf = np.cumsum(src_counts) / src.size 

    ref_cdf = np.cumsum(tmpl_counts) / ref.size

    interp_a_values = np.interp(src_cdf, ref_cdf, np.arange(256))
    return interp_a_values[src.ravel()].reshape(src.shape).astype(np.uint8)

In [ ]:
files = ['org.png', 'noisy_1.jpg', 'noisy_2.jpg', 'noisy_3.jpg']
images = [cv2.resize(cv2.cvtColor(cv2.imread(file), cv2.COLOR_BGR2GRAY), (512, 512), interpolation = cv2.INTER_AREA)\
           for file in files]

fig, axs = plt.subplots(2, 3, figsize=(10, 6))
axs[0, 0].imshow(images[0], cmap='gray')
axs[0, 1].imshow(images[1], cmap='gray')
org_vs_noisy1 = match_histograms(images[1], images[0])
org_vs_noisy1 = np.divide(org_vs_noisy1, 255)
axs[0, 2].imshow(org_vs_noisy1, cmap='gray')

files = ['org.png', 'noisy_1.jpg', 'noisy_2.jpg', 'noisy_3.jpg']
images = [cv2.resize(cv2.cvtColor(cv2.imread(file), cv2.COLOR_BGR2RGB), (512, 512), interpolation = cv2.INTER_AREA)\
           for file in files]

axs[1, 0].imshow(images[0])
axs[1, 1].imshow(images[1])
org_vs_noisy1 = match_histograms(images[1], images[0])
org_vs_noisy1 = np.divide(org_vs_noisy1, 255)
axs[1, 2].imshow(org_vs_noisy1)

plt.setp(axs, xticks=[], yticks=[])
plt.show()

In [ ]:
files = ['org.png', 'noisy_1.jpg', 'noisy_2.jpg', 'noisy_3.jpg']
images = [cv2.imread(file) for file in files]
CDFs = []
M = N = 512
L = 255

for fname, im in zip(files, images):
    print(f"\033[93m{fname}:\033[0m")
    resized = cv2.resize(im, (M, N), interpolation = cv2.INTER_AREA)
    print(f"Original image size: {im.shape}")
    print(f"Resized image size: {resized.shape}")
    vals = resized.mean(axis=2).flatten()
    # cv2.imshow("Resized image", resized)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    fig2, ax2 = plt.subplots()
    b, bins, patches = ax2.hist(vals, 255, label="histogram bins")
    CDFs.append(np.cumsum(b)/(M*N))
    CDF_scaled = CDFs[-1] * np.max(b) / CDFs[-1].max()

    ax2.plot(range(255), CDF_scaled, label="CDF")
    ax2.set_xlim([0, 255])
    ax2.set_title(f"Histogram of {fname}")
    ax2.legend()
    plt.show()
    
    fig, ax = plt.subplots()
    ax.plot(range(255), CDFs[-1], label="CDF")
    ax.set_xlim([0, 255])
    ax.set_title(f"H(k)")
    ax.legend()
    plt.show()

    # Equalization
    transformed_intensity = [int(np.round((L-1) * CDFs[-1][i])) for i in range(len(CDFs[-1]))]
    equalized_hist = np.zeros(255)
    im_map = im

    for i in range(255):
        equalized_hist[transformed_intensity[i]] += b[i]
    # equalized_hist = [int(np.round((L - 1) * equalized_probs[i])) for i in range(255)]
    # for i in range(255):
    #     indexes = np.array(np.where(im == i)).flatten()
    #     im_map[indexes] = i
    fig3, ax3 = plt.subplots()
    ax3.stairs(equalized_hist, np.arange(0, 256, 1), fill=True)
    plt.show()


In [ ]:
def df(img, deb=""):
    values = np.zeros((256))
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            values[np.round(img[i,j])]+=1
                
    return values


def cdf(hist):  
    cdf = np.zeros((256))
    cdf[0] = hist[0]
    for i in range(1, len(hist)):
        cdf[i]= cdf[i-1]+hist[i]
        
    cdf = np.array([ele*255/cdf[-1] for ele in cdf])   
    return cdf

def equalize_image(image):
    equa = cdf(df(image))
    image_equalized = np.zeros_like(image)
    #image_equalized = np.interp(x=image, xp=range(0,256), fp=equa)
    for x in range(image.shape[0]):
        for y in range(image.shape[1]):
            pixel_val = image[x, y]
            image_equalized[x, y] = equa[pixel_val]
            
    return image_equalized

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

eq1 = equalize_image(org)
eqHist1 = cdf(df(eq1))
hist1 = df(eq1)
eq2 = equalize_image(noisy_1)
eqHist2 = cdf(df(eq2))
hist2 = df(eq2)

plt.figure(4)
plt.title('Equalized input image')
plt.imshow(eq1, cmap="gray", vmin=0, vmax=256)
plt.figure(5)
plt.title('Histogram of equalized input image')
plt.plot(hist1)
plt.figure(6)
plt.title('Equalized template image')
plt.imshow(eq2, cmap="gray", vmin=0, vmax=256)
plt.figure(7)
plt.title('Histogram of equalized template image')
plt.plot(hist2)
plt.show()

print(len(eqHist1))
mappedHist = np.zeros_like(hist1)
matched_image = np.zeros_like(org)

for i in range(1, 256):
    if(eqHist2[i] != 0):
        idx = find_nearest(eqHist1, eqHist2[i])
        mappedHist[i] = hist1[idx]
        
match_equa = cdf(mappedHist)
#matched_image = np.interp(x=org, xp=range(0,256), fp=mappedHist)
#cv.imwrite('newimg.jpg', matched_image)
for x in range(matched_image.shape[0]):
    for y in range(matched_image.shape[1]):
        pixel_val = org[x, y]
        matched_image[x, y] = match_equa[pixel_val]
		
plt.figure(9)
plt.title('Matched new image Histogram')
plt.plot(df(matched_image))
plt.figure(10)
plt.title('Matched new image')
plt.imshow(matched_image, cmap="gray")
plt.show()